# Module 05: Serial Implementation - Baseline Performance

**Difficulty**: ⭐⭐

**Estimated Time**: 60 minutes

**Prerequisites**: 
- Basic Python programming
- Understanding of loops and functions
- NumPy basics

## Learning Objectives

By the end of this notebook, you will be able to:
1. Implement serial (non-parallel) versions of computationally intensive problems
2. Use profiling tools to measure performance and identify bottlenecks
3. Establish baseline performance metrics for comparison with parallel implementations
4. Apply code documentation best practices for scientific computing
5. Structure code for easy parallelization in future modules

## 1. Setup and Imports

We'll use several profiling and timing tools to measure performance.

In [ ]:
import numpy as np
import time
import cProfile
import pstats
from io import StringIO
from typing import Tuple, List
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("Setup complete!")
print(f"NumPy version: {np.__version__}")

## 2. Timing Utilities

Before implementing our problems, let's create utilities for accurate timing.

In [ ]:
def time_function(func, *args, num_runs=5, **kwargs):
    """
    Time a function execution with multiple runs for accuracy.
    
    Args:
        func: Function to time
        *args: Positional arguments for func
        num_runs: Number of times to run (default: 5)
        **kwargs: Keyword arguments for func
    
    Returns:
        tuple: (mean_time, std_time, result_from_last_run)
    """
    times = []
    result = None
    
    for i in range(num_runs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        times.append(end - start)
    
    mean_time = np.mean(times)
    std_time = np.std(times)
    
    return mean_time, std_time, result


def print_timing_result(name, mean_time, std_time):
    """
    Print timing results in a formatted way.
    
    Args:
        name: Name of the operation
        mean_time: Mean execution time in seconds
        std_time: Standard deviation of execution time
    """
    print(f"{name}:")
    print(f"  Mean time: {mean_time:.4f} seconds")
    print(f"  Std dev:   {std_time:.4f} seconds")
    print(f"  Range:     [{mean_time - std_time:.4f}, {mean_time + std_time:.4f}] seconds")


# Test the timing utilities
def test_function(n):
    """Simple test function that sums numbers."""
    return sum(range(n))

mean_t, std_t, result = time_function(test_function, 1000000, num_runs=3)
print_timing_result("Test function (sum 1M numbers)", mean_t, std_t)

## 3. Problem 1: Image Processing (Filters)

Image filtering is highly parallelizable because each pixel can be processed independently.
We'll implement Gaussian blur and edge detection filters.

### 3.1 Create Test Images

In [ ]:
def create_test_image(size=(512, 512)):
    """
    Create a test image with various patterns for filter testing.
    
    Args:
        size: Tuple of (height, width)
    
    Returns:
        numpy.ndarray: Test image (grayscale, values 0-255)
    """
    height, width = size
    image = np.zeros((height, width), dtype=np.uint8)
    
    # Create gradient background
    for i in range(height):
        image[i, :] = int(255 * i / height)
    
    # Add some geometric shapes (edges for edge detection)
    # Rectangle
    image[100:200, 100:300] = 200
    
    # Circle
    center_y, center_x = height // 2, width // 2
    radius = 80
    y, x = np.ogrid[:height, :width]
    mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
    image[mask] = 150
    
    # Add noise
    noise = np.random.randint(-20, 20, size=(height, width))
    image = np.clip(image.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    return image


# Create test images of different sizes
small_image = create_test_image((256, 256))
medium_image = create_test_image((512, 512))
large_image = create_test_image((1024, 1024))

# Visualize test image
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(small_image, cmap='gray')
plt.title(f'Small Image ({small_image.shape[0]}x{small_image.shape[1]})')
plt.axis('off')

plt.subplot(132)
plt.imshow(medium_image, cmap='gray')
plt.title(f'Medium Image ({medium_image.shape[0]}x{medium_image.shape[1]})')
plt.axis('off')

plt.subplot(133)
plt.imshow(large_image, cmap='gray')
plt.title(f'Large Image ({large_image.shape[0]}x{large_image.shape[1]})')
plt.axis('off')

plt.tight_layout()
plt.show()

print(f"Created test images: {small_image.shape}, {medium_image.shape}, {large_image.shape}")

### 3.2 Serial Gaussian Blur Implementation

In [ ]:
def create_gaussian_kernel(size=5, sigma=1.0):
    """
    Create a Gaussian kernel for image filtering.
    
    Args:
        size: Kernel size (must be odd)
        sigma: Standard deviation of Gaussian distribution
    
    Returns:
        numpy.ndarray: Normalized Gaussian kernel
    """
    # Ensure size is odd
    if size % 2 == 0:
        size += 1
    
    # Create coordinate arrays
    ax = np.arange(-size // 2 + 1, size // 2 + 1)
    xx, yy = np.meshgrid(ax, ax)
    
    # Calculate Gaussian values
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    
    # Normalize so sum equals 1
    kernel = kernel / np.sum(kernel)
    
    return kernel


def apply_filter_serial(image, kernel):
    """
    Apply a convolution filter to an image using serial processing.
    
    This is the baseline implementation that processes each pixel sequentially.
    
    Args:
        image: Input image (2D numpy array)
        kernel: Convolution kernel (2D numpy array)
    
    Returns:
        numpy.ndarray: Filtered image
    """
    height, width = image.shape
    k_height, k_width = kernel.shape
    
    # Calculate padding needed
    pad_h = k_height // 2
    pad_w = k_width // 2
    
    # Pad image to handle borders
    padded = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='edge')
    
    # Create output image
    output = np.zeros_like(image, dtype=np.float32)
    
    # Apply convolution - THIS IS THE BOTTLENECK
    # Each pixel processed sequentially (nested loops)
    for i in range(height):
        for j in range(width):
            # Extract region of interest
            region = padded[i:i+k_height, j:j+k_width]
            
            # Apply kernel (element-wise multiply and sum)
            output[i, j] = np.sum(region * kernel)
    
    # Clip values to valid range and convert back to uint8
    output = np.clip(output, 0, 255).astype(np.uint8)
    
    return output


# Create Gaussian kernel
gaussian_kernel = create_gaussian_kernel(size=5, sigma=1.5)

print("Gaussian Kernel (5x5):")
print(gaussian_kernel)
print(f"\nKernel sum (should be ~1.0): {np.sum(gaussian_kernel):.6f}")

### 3.3 Test Gaussian Blur and Measure Performance

In [ ]:
# Test on small image first
print("Testing Gaussian blur on small image...")
mean_t, std_t, blurred_small = time_function(
    apply_filter_serial, 
    small_image, 
    gaussian_kernel,
    num_runs=3
)
print_timing_result("Small image (256x256)", mean_t, std_t)

# Visualize result
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.imshow(small_image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(122)
plt.imshow(blurred_small, cmap='gray')
plt.title('Gaussian Blur (Serial)')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Benchmark different image sizes
print("\n" + "="*60)
print("GAUSSIAN BLUR PERFORMANCE BENCHMARK (Serial)")
print("="*60)

benchmark_results = []

test_images = [
    ("Small (256x256)", small_image),
    ("Medium (512x512)", medium_image),
    ("Large (1024x1024)", large_image)
]

for name, img in test_images:
    print(f"\nTesting {name}...")
    mean_t, std_t, _ = time_function(
        apply_filter_serial, 
        img, 
        gaussian_kernel,
        num_runs=3
    )
    print_timing_result(name, mean_t, std_t)
    
    # Calculate pixels processed per second
    pixels = img.shape[0] * img.shape[1]
    pixels_per_sec = pixels / mean_t
    
    benchmark_results.append({
        'Image Size': name,
        'Pixels': pixels,
        'Mean Time (s)': mean_t,
        'Std Dev (s)': std_t,
        'Pixels/sec': pixels_per_sec
    })

# Display results as table
df_results = pd.DataFrame(benchmark_results)
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
print(df_results.to_string(index=False))

### 3.4 Serial Edge Detection Implementation

In [ ]:
def create_sobel_kernels():
    """
    Create Sobel edge detection kernels for horizontal and vertical edges.
    
    Returns:
        tuple: (horizontal_kernel, vertical_kernel)
    """
    # Sobel horizontal (detects vertical edges)
    sobel_x = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ], dtype=np.float32)
    
    # Sobel vertical (detects horizontal edges)
    sobel_y = np.array([
        [-1, -2, -1],
        [ 0,  0,  0],
        [ 1,  2,  1]
    ], dtype=np.float32)
    
    return sobel_x, sobel_y


def edge_detection_serial(image):
    """
    Perform edge detection using Sobel operator (serial implementation).
    
    Args:
        image: Input grayscale image
    
    Returns:
        numpy.ndarray: Edge magnitude image
    """
    sobel_x, sobel_y = create_sobel_kernels()
    
    # Apply both kernels
    edges_x = apply_filter_serial(image, sobel_x)
    edges_y = apply_filter_serial(image, sobel_y)
    
    # Calculate edge magnitude
    magnitude = np.sqrt(edges_x.astype(np.float32)**2 + edges_y.astype(np.float32)**2)
    
    # Normalize to 0-255 range
    magnitude = np.clip(magnitude, 0, 255).astype(np.uint8)
    
    return magnitude


# Test edge detection
print("Testing edge detection on small image...")
mean_t, std_t, edges_small = time_function(
    edge_detection_serial, 
    small_image,
    num_runs=3
)
print_timing_result("Edge detection (256x256)", mean_t, std_t)

# Visualize
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(small_image, cmap='gray')
plt.title('Original')
plt.axis('off')

plt.subplot(132)
plt.imshow(blurred_small, cmap='gray')
plt.title('Gaussian Blur')
plt.axis('off')

plt.subplot(133)
plt.imshow(edges_small, cmap='gray')
plt.title('Edge Detection')
plt.axis('off')

plt.tight_layout()
plt.show()

## 4. Problem 2: Matrix Multiplication

Matrix multiplication is a fundamental operation in scientific computing and machine learning.

In [ ]:
def matrix_multiply_serial(A, B):
    """
    Multiply two matrices using serial triple-nested loop.
    
    This is the naive implementation that serves as our baseline.
    Time complexity: O(n³) for n×n matrices
    
    Args:
        A: First matrix (m × n)
        B: Second matrix (n × p)
    
    Returns:
        numpy.ndarray: Result matrix (m × p)
    """
    m, n = A.shape
    n2, p = B.shape
    
    # Verify dimensions are compatible
    assert n == n2, f"Incompatible dimensions: {A.shape} × {B.shape}"
    
    # Initialize result matrix
    C = np.zeros((m, p), dtype=np.float64)
    
    # Triple nested loop - THIS IS THE BOTTLENECK
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


def matrix_multiply_numpy(A, B):
    """
    Multiply matrices using NumPy's optimized implementation.
    
    This uses BLAS/LAPACK libraries and serves as comparison.
    NumPy's implementation is already highly optimized.
    
    Args:
        A: First matrix
        B: Second matrix
    
    Returns:
        numpy.ndarray: Result matrix
    """
    return np.dot(A, B)


# Create test matrices
small_matrix_a = np.random.rand(100, 100)
small_matrix_b = np.random.rand(100, 100)

medium_matrix_a = np.random.rand(300, 300)
medium_matrix_b = np.random.rand(300, 300)

large_matrix_a = np.random.rand(500, 500)
large_matrix_b = np.random.rand(500, 500)

print("Test matrices created:")
print(f"  Small: {small_matrix_a.shape} × {small_matrix_b.shape}")
print(f"  Medium: {medium_matrix_a.shape} × {medium_matrix_b.shape}")
print(f"  Large: {large_matrix_a.shape} × {large_matrix_b.shape}")

In [ ]:
# Verify correctness first (compare with NumPy)
print("Verifying correctness on small matrix...")
result_serial = matrix_multiply_serial(small_matrix_a, small_matrix_b)
result_numpy = matrix_multiply_numpy(small_matrix_a, small_matrix_b)

# Check if results are close (within floating point precision)
are_close = np.allclose(result_serial, result_numpy, rtol=1e-10)
max_diff = np.max(np.abs(result_serial - result_numpy))

print(f"Results match NumPy: {are_close}")
print(f"Maximum difference: {max_diff:.2e}")

if are_close:
    print("✓ Serial implementation is correct!")
else:
    print("✗ Implementation error detected!")

In [ ]:
# Benchmark matrix multiplication
print("\n" + "="*60)
print("MATRIX MULTIPLICATION PERFORMANCE BENCHMARK")
print("="*60)

matrix_results = []

test_matrices = [
    ("Small (100×100)", small_matrix_a, small_matrix_b),
    ("Medium (300×300)", medium_matrix_a, medium_matrix_b),
    # Large matrix takes too long with serial implementation
    # ("Large (500×500)", large_matrix_a, large_matrix_b)
]

for name, A, B in test_matrices:
    print(f"\nTesting {name}...")
    
    # Serial implementation
    print("  Serial implementation:")
    mean_serial, std_serial, _ = time_function(
        matrix_multiply_serial, A, B, num_runs=3
    )
    print(f"    Mean: {mean_serial:.4f} s, Std: {std_serial:.4f} s")
    
    # NumPy implementation (for comparison)
    print("  NumPy implementation (optimized):")
    mean_numpy, std_numpy, _ = time_function(
        matrix_multiply_numpy, A, B, num_runs=3
    )
    print(f"    Mean: {mean_numpy:.4f} s, Std: {std_numpy:.4f} s")
    
    # Calculate speedup (how much faster is NumPy)
    speedup = mean_serial / mean_numpy
    print(f"  NumPy speedup: {speedup:.2f}x")
    
    # Calculate FLOPS (floating point operations per second)
    # Matrix multiplication requires 2n³ operations for n×n matrices
    n = A.shape[0]
    flops = 2 * n**3
    gflops_serial = (flops / mean_serial) / 1e9
    gflops_numpy = (flops / mean_numpy) / 1e9
    
    matrix_results.append({
        'Matrix Size': name,
        'Serial Time (s)': mean_serial,
        'NumPy Time (s)': mean_numpy,
        'Speedup': speedup,
        'Serial GFLOPS': gflops_serial,
        'NumPy GFLOPS': gflops_numpy
    })

# Display results
df_matrix = pd.DataFrame(matrix_results)
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
print(df_matrix.to_string(index=False))

## 5. Problem 3: Monte Carlo Simulation

Monte Carlo methods use random sampling to solve problems. We'll estimate π and calculate option prices.

### 5.1 Monte Carlo π Estimation

In [ ]:
def estimate_pi_serial(num_samples):
    """
    Estimate π using Monte Carlo method (serial implementation).
    
    Method: Generate random points in unit square [0,1]×[0,1].
    Count how many fall inside the quarter circle (x²+y²≤1).
    π ≈ 4 × (points inside circle) / (total points)
    
    Args:
        num_samples: Number of random points to generate
    
    Returns:
        float: Estimated value of π
    """
    inside_circle = 0
    
    # Generate and test points one by one - THIS IS THE BOTTLENECK
    for _ in range(num_samples):
        # Random point in unit square
        x = np.random.random()
        y = np.random.random()
        
        # Check if inside quarter circle
        if x*x + y*y <= 1.0:
            inside_circle += 1
    
    # Calculate π estimate
    pi_estimate = 4.0 * inside_circle / num_samples
    
    return pi_estimate


def estimate_pi_vectorized(num_samples):
    """
    Estimate π using vectorized NumPy operations.
    
    This is faster than the serial version but not parallel (single thread).
    
    Args:
        num_samples: Number of random points
    
    Returns:
        float: Estimated value of π
    """
    # Generate all points at once (vectorized)
    x = np.random.random(num_samples)
    y = np.random.random(num_samples)
    
    # Check which points are inside circle (vectorized)
    inside_circle = np.sum(x*x + y*y <= 1.0)
    
    # Calculate π estimate
    pi_estimate = 4.0 * inside_circle / num_samples
    
    return pi_estimate


# Test with small sample
print("Testing π estimation...")
pi_est = estimate_pi_serial(10000)
error = abs(pi_est - np.pi)
print(f"Estimated π: {pi_est:.6f}")
print(f"Actual π:    {np.pi:.6f}")
print(f"Error:       {error:.6f} ({100*error/np.pi:.2f}%)")

In [ ]:
# Benchmark π estimation
print("\n" + "="*60)
print("MONTE CARLO π ESTIMATION BENCHMARK")
print("="*60)

pi_results = []

sample_sizes = [
    ("Small", 100_000),
    ("Medium", 1_000_000),
    ("Large", 10_000_000)
]

for name, num_samples in sample_sizes:
    print(f"\nTesting {name} ({num_samples:,} samples)...")
    
    # Serial implementation
    print("  Serial implementation:")
    mean_serial, std_serial, pi_serial = time_function(
        estimate_pi_serial, num_samples, num_runs=3
    )
    print(f"    Time: {mean_serial:.4f} s, π estimate: {pi_serial:.6f}")
    
    # Vectorized implementation
    print("  Vectorized implementation:")
    mean_vec, std_vec, pi_vec = time_function(
        estimate_pi_vectorized, num_samples, num_runs=3
    )
    print(f"    Time: {mean_vec:.4f} s, π estimate: {pi_vec:.6f}")
    
    speedup = mean_serial / mean_vec
    print(f"  Vectorization speedup: {speedup:.2f}x")
    
    pi_results.append({
        'Sample Size': f"{num_samples:,}",
        'Serial Time (s)': mean_serial,
        'Vectorized Time (s)': mean_vec,
        'Speedup': speedup,
        'π Estimate': pi_serial,
        'Error': abs(pi_serial - np.pi)
    })

# Display results
df_pi = pd.DataFrame(pi_results)
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
print(df_pi.to_string(index=False))

### 5.2 Monte Carlo Option Pricing

In [ ]:
def monte_carlo_option_pricing_serial(S0, K, T, r, sigma, num_simulations):
    """
    Price a European call option using Monte Carlo simulation (serial).
    
    Black-Scholes formula for stock price evolution:
    S(T) = S(0) * exp((r - σ²/2)T + σ√T*Z)
    where Z ~ N(0,1)
    
    Args:
        S0: Initial stock price
        K: Strike price
        T: Time to maturity (years)
        r: Risk-free interest rate
        sigma: Volatility
        num_simulations: Number of price paths to simulate
    
    Returns:
        float: Estimated option price
    """
    payoffs = []
    
    # Simulate each price path independently - THIS IS THE BOTTLENECK
    for _ in range(num_simulations):
        # Generate random normal variable
        Z = np.random.standard_normal()
        
        # Calculate stock price at maturity
        ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
        
        # Calculate payoff (max(S-K, 0) for call option)
        payoff = max(ST - K, 0)
        payoffs.append(payoff)
    
    # Discount average payoff to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    
    return option_price


def monte_carlo_option_pricing_vectorized(S0, K, T, r, sigma, num_simulations):
    """
    Price a European call option using vectorized Monte Carlo.
    
    Args:
        S0: Initial stock price
        K: Strike price
        T: Time to maturity
        r: Risk-free rate
        sigma: Volatility
        num_simulations: Number of simulations
    
    Returns:
        float: Estimated option price
    """
    # Generate all random variables at once
    Z = np.random.standard_normal(num_simulations)
    
    # Calculate all final stock prices (vectorized)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate all payoffs (vectorized)
    payoffs = np.maximum(ST - K, 0)
    
    # Discount average payoff
    option_price = np.exp(-r * T) * np.mean(payoffs)
    
    return option_price


# Test option pricing
print("Testing option pricing...")
print("\nOption parameters:")
S0 = 100      # Initial stock price
K = 105       # Strike price
T = 1.0       # 1 year to maturity
r = 0.05      # 5% risk-free rate
sigma = 0.2   # 20% volatility

print(f"  S0 (initial price): ${S0}")
print(f"  K (strike price):   ${K}")
print(f"  T (maturity):       {T} years")
print(f"  r (risk-free rate): {r*100}%")
print(f"  σ (volatility):     {sigma*100}%")

price_serial = monte_carlo_option_pricing_serial(S0, K, T, r, sigma, 10000)
print(f"\nEstimated option price: ${price_serial:.4f}")

In [ ]:
# Benchmark option pricing
print("\n" + "="*60)
print("MONTE CARLO OPTION PRICING BENCHMARK")
print("="*60)

option_results = []

simulation_counts = [
    ("Small", 10_000),
    ("Medium", 100_000),
    ("Large", 1_000_000)
]

for name, num_sims in simulation_counts:
    print(f"\nTesting {name} ({num_sims:,} simulations)...")
    
    # Serial
    print("  Serial implementation:")
    mean_serial, std_serial, price_serial = time_function(
        monte_carlo_option_pricing_serial,
        S0, K, T, r, sigma, num_sims,
        num_runs=3
    )
    print(f"    Time: {mean_serial:.4f} s, Price: ${price_serial:.4f}")
    
    # Vectorized
    print("  Vectorized implementation:")
    mean_vec, std_vec, price_vec = time_function(
        monte_carlo_option_pricing_vectorized,
        S0, K, T, r, sigma, num_sims,
        num_runs=3
    )
    print(f"    Time: {mean_vec:.4f} s, Price: ${price_vec:.4f}")
    
    speedup = mean_serial / mean_vec
    print(f"  Vectorization speedup: {speedup:.2f}x")
    
    option_results.append({
        'Simulations': f"{num_sims:,}",
        'Serial Time (s)': mean_serial,
        'Vectorized Time (s)': mean_vec,
        'Speedup': speedup,
        'Option Price': price_serial
    })

# Display results
df_option = pd.DataFrame(option_results)
print("\n" + "="*60)
print("SUMMARY TABLE")
print("="*60)
print(df_option.to_string(index=False))

## 6. Profiling with cProfile

Let's use Python's built-in profiler to identify bottlenecks in detail.

In [ ]:
def profile_function(func, *args, **kwargs):
    """
    Profile a function using cProfile and display results.
    
    Args:
        func: Function to profile
        *args: Arguments for function
        **kwargs: Keyword arguments for function
    """
    profiler = cProfile.Profile()
    
    # Run function under profiler
    profiler.enable()
    result = func(*args, **kwargs)
    profiler.disable()
    
    # Get statistics
    s = StringIO()
    stats = pstats.Stats(profiler, stream=s)
    stats.sort_stats('cumulative')
    stats.print_stats(20)  # Top 20 functions
    
    print(s.getvalue())
    
    return result


print("Profiling image filter (small image)...")
print("="*60)
profile_function(apply_filter_serial, small_image, gaussian_kernel)

In [ ]:
print("\nProfiling matrix multiplication (small matrix)...")
print("="*60)
profile_function(matrix_multiply_serial, small_matrix_a, small_matrix_b)

In [ ]:
print("\nProfiling Monte Carlo simulation (100K samples)...")
print("="*60)
profile_function(estimate_pi_serial, 100000)

## 7. Summary: Baseline Performance Results

Let's consolidate all our baseline performance measurements.

In [ ]:
print("="*70)
print("BASELINE PERFORMANCE SUMMARY (SERIAL IMPLEMENTATIONS)")
print("="*70)

print("\n1. IMAGE PROCESSING (Gaussian Blur)")
print("-" * 70)
print(df_results.to_string(index=False))

print("\n\n2. MATRIX MULTIPLICATION")
print("-" * 70)
print(df_matrix.to_string(index=False))

print("\n\n3. MONTE CARLO π ESTIMATION")
print("-" * 70)
print(df_pi.to_string(index=False))

print("\n\n4. MONTE CARLO OPTION PRICING")
print("-" * 70)
print(df_option.to_string(index=False))

print("\n" + "="*70)
print("KEY INSIGHTS:")
print("="*70)
print("""
1. IMAGE PROCESSING: O(n²) complexity - scales quadratically with image size
   - Each pixel requires convolution with kernel
   - Highly parallelizable (pixels are independent)

2. MATRIX MULTIPLICATION: O(n³) complexity - scales cubically
   - Triple nested loop is the bottleneck
   - NumPy is 10-100x faster (using optimized BLAS)
   - Excellent candidate for parallelization

3. MONTE CARLO: O(n) complexity - linear with sample size
   - Each sample is independent (embarrassingly parallel)
   - Vectorization gives 10-50x speedup
   - Perfect candidate for GPU acceleration

NEXT STEPS:
- Module 06: Implement multi-core CPU parallelization
- Module 07: Implement GPU acceleration (CUDA)
- Module 08: Comprehensive performance benchmarking
- Module 09: Advanced optimization techniques
""")

## 8. Exercises

Practice implementing and profiling serial algorithms.

### Exercise 1: Implement Median Filter

Implement a median filter for noise removal. Instead of weighted average (like Gaussian), use the median value of the neighborhood.

**Task**: Complete the function below and benchmark it.

In [ ]:
def median_filter_serial(image, kernel_size=5):
    """
    Apply median filter to remove noise.
    
    Args:
        image: Input image
        kernel_size: Size of filter window (must be odd)
    
    Returns:
        numpy.ndarray: Filtered image
    """
    # TODO: Implement median filter
    # Hint: Use np.median() on the neighborhood
    pass

# Test your implementation
# filtered = median_filter_serial(small_image, kernel_size=5)
# plt.imshow(filtered, cmap='gray')
# plt.title('Median Filtered')
# plt.show()

### Exercise 2: Matrix Multiplication with Different Loop Orders

Matrix multiplication has 6 possible loop orderings (ijk, ikj, jik, jki, kij, kji). Try different orders and measure performance.

**Task**: Why does loop order matter? (Hint: cache locality)

In [ ]:
def matrix_multiply_ikj(A, B):
    """
    Matrix multiplication with i-k-j loop order.
    """
    m, n = A.shape
    n2, p = B.shape
    C = np.zeros((m, p), dtype=np.float64)
    
    # TODO: Implement i-k-j loop order
    # for i in range(m):
    #     for k in range(n):
    #         for j in range(p):
    #             ...
    
    return C

# Benchmark different loop orders
# Compare: ijk, ikj, jik, jki, kij, kji

### Exercise 3: Monte Carlo Integration

Use Monte Carlo to estimate the integral of f(x) = x² from 0 to 1.

**Analytical answer**: ∫₀¹ x² dx = 1/3 ≈ 0.333333

In [ ]:
def monte_carlo_integrate(func, a, b, num_samples):
    """
    Estimate integral of func from a to b using Monte Carlo.
    
    Method: Average value × interval length
    ∫ₐᵇ f(x)dx ≈ (b-a) × mean(f(random points))
    
    Args:
        func: Function to integrate
        a: Lower bound
        b: Upper bound
        num_samples: Number of random samples
    
    Returns:
        float: Estimated integral
    """
    # TODO: Implement Monte Carlo integration
    pass

# Test
# def f(x):
#     return x**2
# 
# estimate = monte_carlo_integrate(f, 0, 1, 1000000)
# print(f"Estimated: {estimate:.6f}")
# print(f"Actual:    {1/3:.6f}")
# print(f"Error:     {abs(estimate - 1/3):.6f}")

## Summary

In this module, you learned:

1. **Serial Implementation Patterns**
   - Image processing (2D convolution)
   - Matrix operations (dense linear algebra)
   - Monte Carlo simulations (random sampling)

2. **Performance Profiling**
   - Timing with `time.perf_counter()`
   - Statistical analysis (mean, std dev)
   - Bottleneck identification with cProfile

3. **Baseline Metrics**
   - Execution time measurements
   - Throughput calculations (pixels/sec, GFLOPS)
   - Comparison with optimized libraries (NumPy)

4. **Code Structure for Parallelization**
   - Independent iterations (perfect for parallelization)
   - Data dependencies (require careful handling)
   - Vectorization opportunities

**What's Next?**

- **Module 06**: Multi-core CPU parallelization with multiprocessing
- **Module 07**: GPU acceleration with Numba CUDA
- **Module 08**: Comprehensive benchmarking and analysis

**Additional Resources**

- Python profiling: https://docs.python.org/3/library/profile.html
- NumPy performance tips: https://numpy.org/doc/stable/user/performance.html
- Algorithm complexity: https://www.bigocheatsheet.com/